<a href="https://colab.research.google.com/github/georgetegral/ModeloRepositorioJusticIA/blob/main/JusticIA_AccesoDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

in_dir = 'drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/'
in_file = 'JusticIA_DatosTranscripciones.csv'

df = pd.read_csv(in_dir+in_file)
df.tail()

,Fuente,Conjunto,NombreArchivo,Texto,MetodoTexto
2954,Expedientes AGN,Expedientes,AGN_DFS_TARJETAS_Morales_Lopez_Delia_Cirila_pg...,NaN,NaN
2955,Expedientes AGN,Expedientes,AGN_DFS_TARJETAS_Morales_Lopez_Delia_Cirila_pg...,NaN,NaN
2956,Expedientes AGN,Expedientes,AGN_DFS_TARJETAS_Morales_Lopez_Delia_Cirila_pg...,NaN,NaN
2957,Expedientes AGN,Expedientes,AGN_DFS_TARJETAS_Morales_Lopez_Delia_Cirila_pg...,NaN,NaN
2958,Expedientes AGN,Expedientes,AGN_DFS_TARJETAS_Morales_Lopez_Delia_Cirila_pg...,NaN,NaN


In [ ]:
import PIL

from glob import glob
from random import sample

def get_width_and_height(file_name):
    image = PIL.Image.open(file_name)
    return image.size
    
files = glob(in_dir+'/*/*.*')
files = sample(files, 20)

for file_name in files:
    width, height = get_width_and_height(file_name)
    print(width, height, file_name)

4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_manual/Ficheros_UAG_76-04-05_a_76-11-15_UAG,_fichas-94-.JPG
3456 2592 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Expedientes/AGN_DFS_C.18_2_de_2_Cabrera_Arenas_Emma_pg-279.jpg
4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_auto/Ficheros_Represores_Figueroa_Figueroa_Ruben_Figueroa_Figueroa_Ruben_2_De_79-07-20_a_83-06-04,_FIGUEROA_FIGUEROA,_Ruben-65-.JPG
2304 1728 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_manual/Ficheros_Brigada_Campesina_de_Ajusticiamiento_BCA_Comisiones_Brigada_Campesina_de_Ajusticiamiento,_Comisiones-89-.JPG
1920 1080 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Expedientes/AGN_DFS_C.18_1_de_2_Cabrera_Arenas_Emma_pg-341.jpg
2304 1728 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_manual/Ficheros_Detenidos

In [ ]:
df_long = df[df['Texto'].str.len()>2000]

for dir, fil in zip(df_long['Conjunto'].values, df_long['NombreArchivo'].values):
    file_name = in_dir + dir + '/' + fil
    width, height = get_width_and_height(file_name)
    print(width, height, file_name)

3456 4608 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_manual/Ficheros_UPCEG,_Version_Publica-59-04-09_a_85-12-04-,_Guerrero_UPCEG,_2_de_4-67-04-21_a_69-09-18-_2_de_4-67-04-21_a_69-09-18-33-.JPG
4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_auto/Ficheros_Corral_Garcia_Salvador,_Version_Publica_Corral_Garcia_Salvador,_Version_Publica-147-.JPG
4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_auto/Ficheros_Detenidos_Desaparecidos_Arenas_Bautista_Marcelo_74-08-09,_ARENAS_BAUTISTA,_Marcelo-10-.JPG
4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_auto/Ficheros_Esparza_Flores_Jose_Luis_Esparza_Flores_Jose_Luis-23-.JPG
4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackathon/Datos - Hackathon JusticIA/Fichas_auto/Ficheros_Esparza_Flores_Jose_Luis_Esparza_Flores_Jose_Luis-26-.JPG
4608 3456 drive/MyDrive/Colab Notebooks/RII AA Hackat

In [ ]:
!pip install pdf2image

In [ ]:
import os
from PIL import Image
from distutils.dir_util import copy_tree
from pdf2image import convert_from_path
from math import ceil, floor
import requests
import json
import csv
import time

#path =r'./imgtmp/'
path =r'drive/MyDrive/Colab Notebooks/RII AA Hackathon/imgtmp/'
path_2=r'drive/MyDrive/Colab Notebooks/RII AA Hackathon/imgresult/'
list_of_files = []

def copyImages(fromDirectory,toDirectory):
  copy_tree(fromDirectory, toDirectory)


#Function to check if returned JSON has all the necessary fields in each nested field of the JSON
def checkValueField(rs,field,i,var):
    val = ""
    try:
        val = rs[field][i][var]
    except ValueError:
        val = ""
    except KeyError:
        val = ""
    finally:
        return val

def convertPdf(path, file):
    s = time.perf_counter()
    fullpath = path + file
    images = convert_from_path(fullpath)
    name = file.split('.pdf')[0]
    nameJpg = name +'.jpg'
    for i in range(len(images)):
        # Save pages as images in the pdf
        images[i].save(path + nameJpg, 'JPEG')
    print(f'convertPdf in {time.perf_counter() - s:0.2f} seconds.')
    return nameJpg

def orientationModel(path, file):
    s = time.perf_counter()
    #Request to orientation model
    with open(path+file, 'rb') as f:
        data = f.read()
    req = requests.post(
        url="https://cscvriia-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/2bb8fee1-6413-418c-9c6e-3c41629397ce/classify/iterations/Iteration2/image",
        headers={
            "Content-Type": "application/octet-stream",
            "Prediction-Key": "bac8aef998f94f04b527450124a7864a"
        },
        data = data
    )
    result = json.loads(req.content)
    tag = result['predictions'][0]['tagName']
    print(f'orientationModel in {time.perf_counter() - s:0.2f} seconds.')
    return int(tag)

def removeExif(path, file):
    s = time.perf_counter()
    fullpath = path + file
    image = Image.open(fullpath)
    # next 4 lines strip exif metadata
    data = list(image.getdata())
    im = Image.new(image.mode, image.size)
    im.putdata(data)
    im.save(fullpath)
    print(f'removeExif in {time.perf_counter() - s:0.2f} seconds.')

def rotateFile(path, file, orientation):
    s = time.perf_counter()
    fullpath = path + file
    im = Image.open(fullpath)

    if orientation == 90:
        im = im.transpose(Image.ROTATE_90)
        im.save(fullpath,'JPEG')

    elif orientation == 180:
        im = im.transpose(Image.ROTATE_180)
        im.save(fullpath,'JPEG')

    elif orientation == 270:
        im = im.transpose(Image.ROTATE_270)
        im.save(fullpath,'JPEG')
    print(f'rotateFile in {time.perf_counter() - s:0.2f} seconds.')

def getImageSize(path, file):
    im = Image.open(path+file)
    return int(im.width),int(im.height)

def roundNum(num):
    decimal = num % 1
    if decimal >= .5:
        return ceil(num)
    else:
        return floor(num)

def calculateBoundingBox(left, top, width, height, widthImg, heightImg):
    #https://stackoverflow.com/questions/53737055/bounding-box-left-top-height-width-to-php-x1-x2-y1-y2-coordinates
    w = widthImg *  width
    h = heightImg * height

    xmin = left * widthImg
    ymin = top * heightImg
    xmax = xmin + w
    ymax = ymin + h

    return roundNum(xmin), roundNum(ymin), roundNum(xmax), roundNum(ymax)

def objectDetectionModel(path, file, width, height):
    s = time.perf_counter()
    #request to Object Detection Model
    with open(path+file, 'rb') as f:
        data = f.read()
    req = requests.post(
        url="https://cscvriia-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/c2380253-45d3-4804-9c76-093802973de6/detect/iterations/Iteration2/image",
        headers={
            "Content-Type": "application/octet-stream",
            "Prediction-Key": "bac8aef998f94f04b527450124a7864a"
        },
        data = data
    )
    result = json.loads(req.content)
    finalResult = {}
    #Ignore probability below 0.5
    i = 0
    acum = 0
    while checkValueField(result,'predictions',i,'probability'):
        probability = checkValueField(result,'predictions',i,'probability')
        if probability >= 0.5:
            tagName = checkValueField(result,'predictions',i,'tagName')
            boundingBoxLeft = result['predictions'][i]['boundingBox']['left']
            boundingBoxTop = result['predictions'][i]['boundingBox']['top']
            boundingBoxWidth = result['predictions'][i]['boundingBox']['width']
            boundingBoxHeight = result['predictions'][i]['boundingBox']['height']
            xmin, ymin, xmax, ymax = calculateBoundingBox(boundingBoxLeft, boundingBoxTop, boundingBoxWidth, boundingBoxHeight, width, height)
            finalResult[acum] = [tagName, probability, xmin, ymin, xmax, ymax]
            acum += 1
        
        i += 1

    print(f'objectDetectionModel in {time.perf_counter() - s:0.2f} seconds.')
    return finalResult

def createCsv(prefix):
    with open(prefix+'results.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['filename','width','height','class','xmin','ymin','xmax','ymax'])

def appendCsv(filename, width, height, result,prefix):
    with open(prefix+'results.csv', 'a+', newline='') as file:
        writer = csv.writer(file)
        #For loop each result
        for i in result:
            writer.writerow([filename, width, height, result[i][0], result[i][2], result[i][3], result[i][4], result[i][5]])

for root, dirs, files in os.walk(path):
    for file in files:
        if file == '.DS_Store':
            continue
        list_of_files.append(os.path.join(file))

s = time.perf_counter()

#Limpiar Fotos
for file in list_of_files:
    removeExif(path, file)
#Copiar imagenes de folder 1 a folder 2
copyImages(path,path_2)
prefix = path_2
createCsv(prefix)

for file in list_of_files:
    #Si el archivo es pdf, convertirlo a jpg
    if file.endswith('.pdf'):
        file = convertPdf(prefix, file)
    #Obtener la orientacion del archivo
    orientation = orientationModel(prefix, file)
    #Rotar la imagen
    rotateFile(prefix, file, orientation)
    #Obtener ancho y largo de la foto
    width, height = getImageSize(prefix, file)
    #Llamar al modelo de detección de objetos
    result = objectDetectionModel(prefix, file, width, height)
    #Adjuntar a un csv
    appendCsv(file, width, height, result,prefix)
    print(file)
    print("----------")


print(f'finished in {time.perf_counter() - s:0.2f} seconds.')

removeExif in 0.25 seconds.
orientationModel in 0.88 seconds.
rotateFile in 0.05 seconds.
objectDetectionModel in 0.80 seconds.
derecha.jpeg
----------
finished in 2.05 seconds.
